# install

In [1]:
# !pip install torch
!pip install torchvision==0.5.0
!pip install scipy==1.4.1
!pip install gdown
!pip install tensorboard

You should consider upgrading via the '/local/python-3.7.9/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/local/python-3.7.9/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/local/python-3.7.9/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/local/python-3.7.9/bin/python3.7 -m pip install --upgrade pip' command.


# import

In [2]:
import numpy as np
from PIL.Image import fromarray

import torch
import torchreid
from torchreid.data.transforms import build_transforms

/workspace/notebook/sandbox/deep-person-reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '


In [3]:
class BaseModel():
    height = 256
    width = 128
    
    def __init__(self, name='osnet_x1_0'):
    # def __init__(self, name='osnet_ain_x1_0'):
        self.name = name
        self._set_model(name)
        self._set_transformer()

    def _set_model(self, name):
        model = torchreid.models.build_model(
            name=name, # This can be:  'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_25', 'resnet18'
            num_classes=751,
            pretrained=True
        )
        model = model.cuda()
        self.reid_model = model.eval()
    
    def _set_transformer(self):
        _, self.transformer = build_transforms(
                self.height,
                self.width,)
    
    def compute(self, images):
        images = self._validate(images)
        transformed_images = self._pre_process(images)
        results = self._inference(transformed_images)
        outputs = self._post_process(results)
        return outputs
    
    def _validate(self, images):
        if isinstance(images, list):
            pass

        elif isinstance(images, np.ndarray):
            images = [images]
        
        else:
            raise ValueError
        return images
        

    def _pre_process(self, images):
        transformed_images = []
        for image in images:
            image = fromarray(image)
            transformed_image = self.transformer(image)
            transformed_image = transformed_image.reshape(1, 3, self.height, self.width)
            transformed_images.append(transformed_image)
        transformed_images = torch.cat(transformed_images).cuda()
        return transformed_images
        
    def _inference(self, images):
        results = self.reid_model(images)
        return results

    def _post_process(self, results):
        outputs = results.cpu().detach().numpy()
        return outputs

In [4]:
model_name = "osnet_x0_25"
model = torchreid.models.build_model(
    name=model_name, # This can be:  'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_25', 'resnet18'
    num_classes=1000,
    pretrained=True
)

Successfully loaded imagenet pretrained weights from "/home/ubuntu/.cache/torch/checkpoints/osnet_x0_25_imagenet.pth"


In [5]:
model.eval()

OSNet(
  (conv1): ConvLayer(
    (conv): Conv2d(3, 16, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): Sequential(
    (0): OSBlock(
      (conv1): Conv1x1(
        (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (conv2a): LightConv3x3(
        (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (conv2b): Sequential(
        (

# convert

In [6]:
import torch.onnx

In [7]:
batch_size = 3
height = 256
width = 128

In [8]:
x = torch.randn(batch_size, 3, height, width, requires_grad=True)

In [9]:
torch_out = model(x)

In [10]:
model_save_path = "{}.onnx".format(model_name)

In [11]:
convert = False
if convert:
    torch.onnx.export(model,               # model being run
                      x,                         # model input (or a tuple for multiple inputs)
                      model_save_path,   # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})


# onnx

In [12]:
import onnx

onnx_model = onnx.load(model_save_path)
onnx.checker.check_model(onnx_model)

In [13]:
import onnxruntime

sess = onnxruntime.InferenceSession(model_save_path)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {sess.get_inputs()[0].name: to_numpy(x)}
ort_outs = sess.run(None, ort_inputs)

In [14]:
ort_outs

[array([[0.        , 0.        , 0.        , ..., 0.        , 0.48593774,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.26337618,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.84350514,
         0.        ]], dtype=float32)]

In [15]:
# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), 
                           ort_outs[0], 
                           rtol=1e-03, 
                           atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


# pre process

## constant

In [89]:
norm_mean = np.array([0.485, 0.456, 0.406])
norm_std = np.array([0.229, 0.224, 0.225])

## data

In [293]:
import cv2

In [296]:
img = cv2.imread('/workspace/data/dog.jpg')
# img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
# img = cv2.resize(img, (height, width), cv2.INTER_LINEAR)

## torch

In [297]:
from torchvision.transforms import (
    Resize, Compose, ToTensor, Normalize, ColorJitter, RandomHorizontalFlip, CenterCrop
)

In [298]:
height = 256
width = 128
normalize = Normalize(mean=norm_mean, std=norm_std)

In [325]:
transform = Compose([
        Resize((height, width)),
        ToTensor(),
        normalize,
    ])

In [326]:
transform

Compose(
    Resize(size=(256, 128), interpolation=PIL.Image.BILINEAR)
    ToTensor()
    Normalize(mean=[0.485 0.456 0.406], std=[0.229 0.224 0.225])
)

In [346]:
from PIL import Image

In [347]:
img_pil = Image.fromarray(img)

In [348]:
trns_img = transform(img_pil)

In [354]:
resize = Resize((height, width))

In [358]:
trns_rsz_img = np.asarray(resize(img_pil))
trns_rsz_img

array([[[ 53,  61,  60],
        [ 51,  59,  58],
        [ 52,  60,  59],
        ...,
        [162, 189, 174],
        [157, 218, 205],
        [ 92, 157, 152]],

       [[ 53,  61,  60],
        [ 53,  61,  60],
        [ 53,  61,  60],
        ...,
        [148, 195, 166],
        [138, 213, 182],
        [ 84, 141, 134]],

       [[ 53,  61,  60],
        [ 53,  61,  60],
        [ 53,  61,  60],
        ...,
        [159, 224, 204],
        [132, 215, 182],
        [ 80, 128, 121]],

       ...,

       [[175, 164, 157],
        [174, 163, 157],
        [177, 166, 159],
        ...,
        [121, 127, 126],
        [122, 127, 127],
        [104, 109, 115]],

       [[177, 166, 158],
        [176, 165, 157],
        [177, 165, 158],
        ...,
        [122, 128, 127],
        [121, 127, 127],
        [ 93,  96, 106]],

       [[175, 165, 156],
        [176, 165, 157],
        [175, 164, 156],
        ...,
        [122, 128, 127],
        [121, 126, 127],
        [ 86,  88,  99]]

In [330]:
np.array(trns_img)[0][0]

array([-1.2102921 , -1.2445416 , -1.2274169 , -1.2102921 , -1.2102921 ,
       -1.1589178 , -0.5424266 , -0.6451751 , -0.31980482, -0.08005828,
       -0.25130582, -0.19993155, -0.6622999 , -0.5424266 , -0.33692956,
       -0.55955136, -0.2855553 , -0.49105233, -0.43967807,  0.38231018,
       -0.04580877, -0.18280679, -0.57667613, -0.5424266 ,  0.07406463,
       -0.14855729, -0.23418105, -0.23418105, -0.45680285, -0.18280679,
       -0.35405433,  0.5535577 ,  0.15968838, -0.04580877,  0.10831413,
       -0.14855729,  0.02269024, -0.2855553 , -0.4739276 , -0.8506721 ,
       -0.57667613, -0.30268008,  0.00556549, -0.38830382, -0.79929787,
       -0.9362959 , -0.8677969 , -0.4739276 , -0.8335474 , -0.04580877,
       -0.18280679, -0.49105233, -0.81642264,  0.4850587 ,  0.72480524,
        0.15968838, -0.16568205, -0.37117907, -1.1589178 , -1.0904187 ,
       -1.2102921 , -1.1760426 , -1.0904187 , -1.3130406 , -1.3130406 ,
       -1.2445416 , -1.2616663 , -1.2102921 , -1.2274169 , -1.19

In [331]:
trns_img.shape
trns_img[0]

tensor([[-1.2103, -1.2445, -1.2274,  ...,  0.6563,  0.5707, -0.5424],
        [-1.2103, -1.2103, -1.2103,  ...,  0.4166,  0.2453, -0.6794],
        [-1.2103, -1.2103, -1.2103,  ...,  0.6049,  0.1426, -0.7479],
        ...,
        [ 0.8789,  0.8618,  0.9132,  ..., -0.0458, -0.0287, -0.3369],
        [ 0.9132,  0.8961,  0.9132,  ..., -0.0287, -0.0458, -0.5253],
        [ 0.8789,  0.8961,  0.8789,  ..., -0.0287, -0.0458, -0.6452]])

In [373]:
def resize_compatibility_with_torchvision(img,
                                          height, 
                                          width):
    img_pil = Image.fromarray(img)
    img_resize = img_pil.resize((width, height), 
                                resample=Image.BILINEAR)
    img_resize = np.asarray(img_resize)
    return img_resize

In [378]:
def normalize_compatibility_with_torchvision(img,
                                          norm_mean=np.array([0.485, 0.456, 0.406]),
                                          norm_std=np.array([0.229, 0.224, 0.225])):
    img_resize = (img - 255 * norm_mean) / (255 * norm_std)
    img_resize = img_resize.transpose(2,0,1)
    img_resize = img_resize.astype(np.float32)
    return img_resize

In [379]:
rsz_img = resize_compatibility_with_torchvision(img, height, width)

In [380]:
norm_img = normalize_compatibility_with_torchvision(rsz_img)

In [383]:
trns_rsz_img[0]

array([[ 53,  61,  60],
       [ 51,  59,  58],
       [ 52,  60,  59],
       [ 53,  61,  60],
       [ 53,  64,  62],
       [ 56,  67,  66],
       [ 92, 135, 128],
       [ 86, 154, 141],
       [105, 177, 153],
       [119, 187, 155],
       [109, 163, 146],
       [112, 158, 149],
       [ 85, 146, 123],
       [ 92, 133, 121],
       [104, 149, 139],
       [ 91, 151, 135],
       [107, 157, 148],
       [ 95, 135, 121],
       [ 98, 145, 139],
       [146, 179, 191],
       [121, 179, 177],
       [113, 180, 165],
       [ 90, 150, 129],
       [ 92, 142, 135],
       [128, 177, 172],
       [115, 166, 168],
       [110, 185, 154],
       [110, 183, 150],
       [ 97, 193, 156],
       [113, 213, 180],
       [103, 202, 168],
       [156, 235, 207],
       [133, 227, 193],
       [121, 214, 186],
       [130, 216, 191],
       [115, 191, 166],
       [125, 188, 171],
       [107, 184, 156],
       [ 96, 171, 135],
       [ 74, 125, 113],
       [ 90, 133, 136],
       [106, 162

In [384]:
rsz_img[0]

array([[ 53,  61,  60],
       [ 51,  59,  58],
       [ 52,  60,  59],
       [ 53,  61,  60],
       [ 53,  64,  62],
       [ 56,  67,  66],
       [ 92, 135, 128],
       [ 86, 154, 141],
       [105, 177, 153],
       [119, 187, 155],
       [109, 163, 146],
       [112, 158, 149],
       [ 85, 146, 123],
       [ 92, 133, 121],
       [104, 149, 139],
       [ 91, 151, 135],
       [107, 157, 148],
       [ 95, 135, 121],
       [ 98, 145, 139],
       [146, 179, 191],
       [121, 179, 177],
       [113, 180, 165],
       [ 90, 150, 129],
       [ 92, 142, 135],
       [128, 177, 172],
       [115, 166, 168],
       [110, 185, 154],
       [110, 183, 150],
       [ 97, 193, 156],
       [113, 213, 180],
       [103, 202, 168],
       [156, 235, 207],
       [133, 227, 193],
       [121, 214, 186],
       [130, 216, 191],
       [115, 191, 166],
       [125, 188, 171],
       [107, 184, 156],
       [ 96, 171, 135],
       [ 74, 125, 113],
       [ 90, 133, 136],
       [106, 162

In [ ]:
trns_rsz_img

In [385]:
np.testing.assert_allclose(trns_rsz_img, 
                           rsz_img, 
                           rtol=1e-03, 
                           atol=1e-05)

In [ ]:
trns_rsz_img

In [340]:
img_resize[0]

array([[-1.210292  , -1.2445415 , -1.2274168 , ...,  0.6563062 ,
         0.5706824 , -0.5424266 ],
       [-1.210292  , -1.210292  , -1.210292  , ...,  0.41655964,
         0.2453121 , -0.6794246 ],
       [-1.210292  , -1.210292  , -1.210292  , ...,  0.60493195,
         0.14256358, -0.7479236 ],
       ...,
       [ 0.878928  ,  0.86180323,  0.9131775 , ..., -0.04580872,
        -0.02868396, -0.33692953],
       [ 0.9131775 ,  0.8960527 ,  0.9131775 , ..., -0.02868396,
        -0.04580872, -0.5253018 ],
       [ 0.878928  ,  0.8960527 ,  0.878928  , ..., -0.02868396,
        -0.04580872, -0.6451751 ]], dtype=float32)

In [356]:
np.testing()

<module 'numpy.testing' from '/home/ubuntu/.local/lib/python3.7/site-packages/numpy/testing/__init__.py'>

In [256]:
norm_img_numpy_trans[0]

array([[-1.24454148, -1.19316722, -1.22741673, ..., -0.08005822,
         0.31381111, -1.33016525],
       [-1.21029198, -1.17604247, -1.17604247, ..., -0.1999315 ,
        -0.04580872, -1.29591575],
       [-1.22741673, -1.19316722, -1.17604247, ..., -1.5356623 ,
        -1.33016525, -1.46716328],
       ...,
       [ 0.89605274,  0.96455176,  1.01592602, ..., -0.02868396,
        -0.02868396, -0.06293347],
       [ 0.96455176,  0.84467848,  0.94742701, ..., -0.04580872,
        -0.02868396, -0.57667609],
       [ 0.94742701,  0.84467848,  0.86180324, ..., -0.01155921,
        -0.04580872, -1.50141279]])

In [116]:
x = torch.randn(batch_size, 3, height, width, requires_grad=True)

In [120]:
x.sub_(torch.to(norm_mean))

TypeError: sub_(): argument 'other' (position 1) must be Tensor, not numpy.ndarray

In [81]:
# from torchreid.data.transforms import build_transforms

# _, transformer = build_transforms(
#                 height,
#                 width,)

# transformer

# trns_img_torchreid = transformer(img_pil)

# trns_img[0]

# trns_img_torchreid[0]

## numpy

In [31]:
norm_mean = np.array([0.4914, 0.4822, 0.4465])
norm_std = np.array([0.229, 0.224, 0.225],)

In [40]:
norm_img_numpy = (img - 255 * norm_mean) / (255 * norm_std)
norm_img_numpy[0]

array([[-1.28961384, -1.13727241, -0.99098039],
       [-1.27248908, -1.11976541, -0.9735512 ],
       [-1.23823958, -1.0847514 , -0.93869281],
       ...,
       [-1.3409881 , -0.59455532,  0.49050109],
       [-1.44373662, -1.27732843, -0.45067538],
       [-1.3409881 , -0.90968137, -0.88640523]])

In [26]:
img.shape

(576, 768, 3)

In [28]:
img

array([[[ 50,  58,  57],
        [ 51,  59,  58],
        [ 53,  61,  60],
        ...,
        [ 47,  89, 142],
        [ 41,  50,  88],
        [ 47,  71,  63]],

       [[ 51,  59,  58],
        [ 51,  59,  58],
        [ 52,  60,  59],
        ...,
        [ 37,  74, 124],
        [ 41,  50,  84],
        [ 46,  70,  58]],

       [[ 51,  59,  58],
        [ 51,  59,  58],
        [ 52,  60,  59],
        ...,
        [ 25,  54,  98],
        [ 48,  54,  77],
        [ 43,  62,  45]],

       ...,

       [[179, 168, 160],
        [179, 168, 160],
        [182, 171, 163],
        ...,
        [ 64,  64,  80],
        [ 36,  39,  53],
        [ 48,  53,  62]],

       [[180, 169, 161],
        [179, 168, 160],
        [175, 164, 156],
        ...,
        [ 61,  61,  79],
        [ 37,  40,  54],
        [ 52,  56,  67]],

       [[176, 165, 157],
        [178, 167, 159],
        [172, 161, 153],
        ...,
        [ 62,  62,  80],
        [ 33,  36,  50],
        [ 35,  39,  50]]

In [126]:
x = np.zeros((3, 100, 100))

In [127]:
x.shape

(3, 100, 100)

In [128]:
x = x[np.newaxis]

In [130]:
x.shape

(1, 3, 100, 100)

In [ ]:
# Export the model
torch.onnx.export(torch_model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "super_resolution.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

In [17]:
model.

OSNet(
  (conv1): ConvLayer(
    (conv): Conv2d(3, 16, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): Sequential(
    (0): OSBlock(
      (conv1): Conv1x1(
        (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (conv2a): LightConv3x3(
        (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (conv2b): Sequential(
        (